In [1]:
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings(action='ignore')

In [4]:
train = pd.read_csv('./train.csv')
train.info()
train.columns = train.columns.str.lower()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [5]:
train['survived'].value_counts()

0    549
1    342
Name: survived, dtype: int64

In [9]:
train.groupby('survived')['survived'].count()

survived
0    549
1    342
Name: survived, dtype: int64

In [11]:
train[['pclass','survived']].value_counts().sort_index()

pclass  survived
1       0            80
        1           136
2       0            97
        1            87
3       0           372
        1           119
dtype: int64

In [13]:
train['cabin2'] = train['cabin'].str[:1]
train['cabin2'].head()

0    NaN
1      C
2    NaN
3      C
4    NaN
Name: cabin2, dtype: object

In [18]:
SS = train['embarked'].mode().values[0]

In [19]:
train['embarked'].fillna(SS)
train['embarked']

0      S
1      C
2      S
3      S
4      S
      ..
886    S
887    S
888    S
889    C
890    Q
Name: embarked, Length: 891, dtype: object

In [28]:
train['name_title'] = train['name'].str.extract('\s([\w]+)[.]\s')
train.head()

,passengerid,survived,pclass,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,cabin2,name_title
0,1,0,333,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,NaN,Mr
1,2,1,111,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,C,Mrs
2,3,1,333,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,NaN,Miss
3,4,1,111,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,C,Mrs
4,5,0,333,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,NaN,Mr


In [21]:
train['pclass'] = train['pclass'].replace(3,33)
train['pclass']

0      33
1       1
2      33
3       1
4      33
       ..
886     2
887     1
888    33
889     1
890    33
Name: pclass, Length: 891, dtype: int64

In [22]:
train['pclass'] = train['pclass'].replace([1,2,33],[111,222,333])
train['pclass']

0      333
1      111
2      333
3      111
4      333
      ... 
886    222
887    111
888    333
889    111
890    333
Name: pclass, Length: 891, dtype: int64

In [23]:
dic = {'female':0,'male':1}
train['sex'] = train['sex'].map(dic)
train['sex']

0      1
1      0
2      0
3      0
4      1
      ..
886    1
887    0
888    0
889    1
890    1
Name: sex, Length: 891, dtype: int64

In [25]:
train['sex'] = train['sex'].apply(lambda x: 'male' if x == 1 else 'female')
train['sex']

0        male
1      female
2      female
3      female
4        male
        ...  
886      male
887    female
888    female
889      male
890      male
Name: sex, Length: 891, dtype: object

In [29]:
train['name_title2'] = train['name_title']
train['name_title2'] = train['name_title2'].replace(['Rev','Major','Major','Col','Capt','Sir','Don','Jonkheer'],
                              ['Mr','Mr','Mr','Mr','Mr','Mr','Mr','Mr'])
train['name_title2'] = train['name_title2'].replace(['Mlle','Ms','Lady'],['Miss','Miss','Miss'])
train['name_title2'] = train['name_title2'].replace(['Countess','Mme'],['Mrs','Mrs'])
train['name_title2'].value_counts()

Mr        531
Miss      186
Mrs       127
Master     40
Dr          7
Name: name_title2, dtype: int64

In [35]:
train[train['age'].notna()].groupby('name_title2')['age'].mean()

name_title2
Dr        42.000000
Master     4.574167
Miss      22.020000
Mr        32.891990
Mrs       35.763636
Name: age, dtype: float64

In [36]:
train.groupby('name_title2')['age'].mean()

name_title2
Dr        42.000000
Master     4.574167
Miss      22.020000
Mr        32.891990
Mrs       35.763636
Name: age, dtype: float64

In [37]:
train['age2'] = train.groupby('name_title2')['age'].transform(lambda g : g.fillna(g.mean()))
train['age2']

0      22.00
1      38.00
2      26.00
3      35.00
4      35.00
       ...  
886    27.00
887    19.00
888    22.02
889    26.00
890    32.00
Name: age2, Length: 891, dtype: float64

In [38]:
train['age_gubun'] = train['age2']//10

In [40]:
train.pivot_table(index = 'pclass', values = 'survived', aggfunc = 'sum')

,survived
pclass,
111,136
222,87
333,119


In [42]:
pd.crosstab(index =train['pclass'],columns=train['survived'])

survived,0,1
pclass,,
111,80,136
222,97,87
333,372,119
